# Модуль 5.2 — Кэширование и Rate Limit

**Цель:** сделать вызовы LLM дешевле и стабильнее: кэшировать ответы и ограничивать частоту запросов.

**Что сделаем:**
- включим кэш в памяти и на диске
- увидим ускорение на повторном запросе
- добавим простое ограничение частоты (rate limit)

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  python-dotenv \
  pydantic==2.12.3 \
  requests==2.32.4

## Настройка провайдера (AITunnel)

Укажем ключ и базовый URL для вызовов модели.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## 1) Кэш в памяти

Повторный запрос с тем же промптом вернётся мгновенно.

In [ ]:
import time
from langchain_openai import ChatOpenAI
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

llm = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=128,
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

prompt = "Объясни одним предложением, что такое кэш." 

start = time.perf_counter()
print(llm.invoke(prompt).content)
print("Первый вызов:", round(time.perf_counter() - start, 3), "s")

start = time.perf_counter()
print(llm.invoke(prompt).content)
print("Второй вызов (из кэша):", round(time.perf_counter() - start, 3), "s")

## 2) Кэш на диске (между запусками)

Если ноутбук перезапустить, кэш останется.

In [ ]:
from langchain.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path="./cache/langchain.sqlite"))

start = time.perf_counter()
print(llm.invoke(prompt).content)
print("Вызов с дисковым кэшем:", round(time.perf_counter() - start, 3), "s")

## 3) Rate limit (простая защита)

Ограничим частоту вызовов: не чаще 1 запроса в 2 секунды.

In [ ]:
import time

last_call = 0.0
min_interval_sec = 2.0


def rate_limited_invoke(prompt_text: str):
    global last_call
    now = time.time()
    wait = min_interval_sec - (now - last_call)
    if wait > 0:
        time.sleep(wait)
    last_call = time.time()
    return llm.invoke(prompt_text).content

for i in range(3):
    t0 = time.perf_counter()
    print(rate_limited_invoke(f"Сообщение {i+1}"))
    print("Время:", round(time.perf_counter() - t0, 2), "s")